<a href="https://colab.research.google.com/github/Delcapa/BDA_6th/blob/main/_Project_Dropout_and_BNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms

# Data Preprocessing

- Transforms.ToTensor</br>
Transform Image from HxWxC for [0,255] to CxHxW for [0.0, 1.0]</br>
</br>
- Transforms.Normalize((mean), (variance))</br>
Standardization data into foe range [-1,1]

In [9]:
# torchvision.transforms.ToTensor() class: PIL 이미지 또는 numpy 배열을
# pytorch tensor("Channel x Height x Width")로 변화 시켜주는  클래스입니다.

# transofrms.Normalize(): 각 채널별 평균(mean)을 뺀 뒤 표준편차(std)로 나누어 정규화를 진행
transform = transforms.Compose([transforms.ToTensor(), # 이미지 픽셀 범위 0~1로 조정
                                transforms.Normalize((0.5,),(0.5,)) # (채널평균:0.5, 0.5, 0.5), (채널표준편차: 0.5, 0.5, 0.5)로 정규화
                                # 변환 후 결과 = (픽셀 값-각 채널평균)/각 채널 표준편차

                               ])



In [3]:
data_train = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
data_test = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(data_train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(data_test, batch_size=32, shuffle=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 16001643.91it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 496663.35it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 926983.04it/s] 


Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3273849.25it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



In [4]:
import torch.nn as nn
import torch.nn.functional as F

# Neural Network Architecture

In [ ]:
class MC_Dropout(nn.Module):
    def __init__(self):
        super(MC_Dropout, self).__init__()
        self.pseudo_bnn = nn.Sequential(
            nn.Conv2d(1,32,3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(32,32,3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),

            nn.Conv2d(32,128,3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128,128,3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128,256,3, stride=1, padding=1),
            nn.Conv2d(256,256,3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU()
        )
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(3*3*256, 4096)
        self.fc2 = nn.Linear(4096, 10)

    def forward(self, x):
        x = self.pseudo_bnn(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

    def enable_dropout(self):
        """ Function to explicitly enable dropout during testing """
        for module in self.modules():
            if isinstance(module, nn.Dropout):
                module.train()

# Training and Test

In [ ]:
model = MC_Dropout()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MC_Dropout(
  (pseudo_bnn): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): ReLU()
    (5): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): ReLU()
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=2304, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=10, bias=True)
)

In [ ]:
# Training function
def train(epoch):
    model.train()  # Always in training mode to keep dropout active
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Running training and testing
for epoch in range(1, 6):  # Train for 10 epochs
    train(epoch)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302057
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.170395
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.080898
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.142739
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.147537
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.005346
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.006632
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.028047
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.047281
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.000484
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.339587
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.007476
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.035016
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.091852
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.102595
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.006699
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.062998
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.027048
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.094034
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.033042
Train Epoch: 2

In [ ]:

# # Test function with MC Dropout
# def test():
#     model.enable_dropout()  # Ensure dropout is enabled
#     test_loss = 0
#     correct = 0
#     with torch.no_grad():
#         for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
#             output = model(data)
#             test_loss += criterion(output, target).item()
#             pred = output.argmax(dim=1, keepdim=True)
#             correct += pred.eq(target.view_as(pred)).sum().item()

#             test_loss /= len(test_loader.dataset)
#             accuracy = 100. * correct / len(test_loader.dataset)
#         print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')


In [ ]:
test()

In [ ]:
# Test function with MC Dropout
def test2():
    model.enable_dropout()  # Ensure dropout is enabled
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

            test_loss /= len(test_loader.dataset)
            accuracy = 100. * correct / len(test_loader.dataset)
        print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')


In [ ]:
test2()